# <center> <img src="../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Big Data** </center>
---
### <center> **Autumn 2025** </center>
---
### <center> **Generate schemas dynamically** </center>
---
**Profesor**: Pablo Camarillo Ramirez

# Find the PySpark Installation

In [1]:
import findspark
findspark.init()

In [2]:
from jaime_galindo.spark_utils import SparkUtils

In [3]:
schema = SparkUtils.generate_schema([
    ("name", "string"),
    ("age", "int"),
    ("city", "string")
])

schema

TypeError: ArrayType.__init__() missing 1 required positional argument: 'elementType'